In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import pandas as pd

In [4]:
import sys
!{sys.executable} -m pip install sentence-transformers newsapi-python
#!pip3 install newsapi-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 7.9 MB/s eta 0:00:0031m10.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.1 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━

In [5]:
from newsapi import NewsApiClient

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [8]:
### Initialize NewsAPI client
newsapi = NewsApiClient(api_key='4a55bffaf68044b3acc586adaaadd52e')

### Fetch today's top headlines from NewsAPI
top_headlines = newsapi.get_top_headlines(language='en')

### Extract article content from the top headlines
articles = [article['content'] for article in top_headlines['articles'] if article['content']]

### Filter out articles with empty content
articles = [article for article in articles if article]

In [14]:
len(top_headlines['articles'])

20

In [15]:
### Initialize SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

### Create embeddings for the articles
embeddings = model.encode(articles)

### Cluster the embeddings
num_clusters = 5  # Adjust as needed
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)
clusters = kmeans.labels_

/home/desh/anaconda3/envs/news_venv/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


/home/desh/anaconda3/envs/news_venv/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11030). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/desh/anaconda3/envs/news_venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5)

In [16]:
### Summarize the clusters
for cluster_idx in range(num_clusters):
    cluster_articles = [articles[i] for i, cluster_label in enumerate(clusters) if cluster_label == cluster_idx]
    centroid_idx = pairwise_distances_argmin_min(kmeans.cluster_centers_[cluster_idx].reshape(1, -1), embeddings)[0][0]
    centroid_article = articles[centroid_idx]
    
    print(f"\nCluster {cluster_idx + 1} - Centroid Article: {centroid_article}")
    print("Top Articles:")
    for article in cluster_articles[:3]:
        print(article)


Cluster 1 - Centroid Article: Bihar chief minister Nitish Kumar won a trust vote in the assembly on Monday, capping a dramatic few weeks that saw him switch allies, take oath for a record ninth time, and tensions escalate in the … [+6593 chars]
Top Articles:
Diamondback Energy (FANG) on agreed to buy privately held Endeavor Energy Resources on Monday for $26 billion including debt, creating a new Permian Shale giant and the latest in a flurry of big rece… [+1947 chars]
Bihar chief minister Nitish Kumar won a trust vote in the assembly on Monday, capping a dramatic few weeks that saw him switch allies, take oath for a record ninth time, and tensions escalate in the … [+6593 chars]

Cluster 2 - Centroid Article: HOUSTON (AP) A womans motive for opening fire in celebrity pastor Joel Osteens megachurch was unclear Monday, a day after the shooting sent worshippers rushing for safety in between busy services.
P… [+3980 chars]
Top Articles:
MOUNT EDEN, Bronx (WABC) -- An investigation is und